In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
#Two Year Treasury Yield
relative_path = os.path.join('..', '..',"Data",'Consolidated',"TreasuryYield", "TUQE.csv")
TUY=pd.read_csv(relative_path,index_col=0)
#Ten Year Treasury Yield
relative_path = os.path.join('..', '..',"Data",'Consolidated',"TreasuryYield", "TYQE.csv")
TYY=pd.read_csv(relative_path,index_col=0)

In [3]:
TUY.index = pd.to_datetime(TUY.index)
TYY.index = pd.to_datetime(TYY.index)

In [4]:
#Two Year Treasury Yield - 3 Month Forecast
relative_path = os.path.join('..', '..',"Data",'Consolidated',"Forecasts","IndividualForecasts", "TMTUIndividual.csv")
TMTUI=pd.read_csv(relative_path,index_col=0)

#Two Year Treasury Yield - 6 Month Forecast
relative_path = os.path.join('..', '..',"Data",'Consolidated',"Forecasts","IndividualForecasts", "SMTUIndividual.csv")
SMTUI=pd.read_csv(relative_path,index_col=0)

#Two Year Treasury Yield - 12 Month Forecast
relative_path = os.path.join('..', '..',"Data",'Consolidated',"Forecasts","IndividualForecasts", "TWTUIndividual.csv")
TWTUI=pd.read_csv(relative_path,index_col=0)

#Ten Year Treasury Yield - 3 Month Forecast
relative_path = os.path.join('..', '..',"Data",'Consolidated',"Forecasts","IndividualForecasts", "TMTYIndividual.csv")
TMTYI=pd.read_csv(relative_path,index_col=0)

#Ten Year Treasury Yield - 6 Month Forecast
relative_path = os.path.join('..', '..',"Data",'Consolidated',"Forecasts","IndividualForecasts", "SMTYIndividual.csv")
SMTYI=pd.read_csv(relative_path,index_col=0)

#Ten Year Treasury Yield - 12 Month Forecast
relative_path = os.path.join('..', '..',"Data",'Consolidated',"Forecasts","IndividualForecasts", "TWTYIndividual.csv")
TWTYI=pd.read_csv(relative_path,index_col=0)

In [ ]:
TMTUI

In [6]:
#remove all rows with less than 10 individual forecasts
def num_val(df):
    df.index=pd.to_datetime(df.index)
    #df.shift(1)
    df=df[::-1]
    df=df[df.notna().sum(axis=1)>=10]
    return(df)

In [7]:
TMTUI=num_val(TMTUI)
SMTUI=num_val(SMTUI)
TWTUI=num_val(TWTUI)
TMTYI=num_val(TMTYI)
SMTYI=num_val(SMTYI)
TWTYI=num_val(TWTYI)

In [8]:
def qn_predictions(individual_forecasts,maturity,tyield,q):
    #determine sign change of yield over q quarters
    actual_change=np.sign(tyield.diff())
    #determine predicted sign change of yield
    pred=np.sign(individual_forecasts.sub(tyield[maturity].reindex(individual_forecasts.index),axis=0)).shift(q)
    #for each prediction, check wheter predicted sign change was correct
    new_df=pd.DataFrame(columns=pred.columns)
    for index, row in pred.iloc[:-1].iterrows():
        #get actual sign change in yield
        act_val=actual_change.loc[index,maturity]
        #go through every column and check wheter sign change was predicted correctly
        for column in pred.columns:
            #if prediction was correct, set value to 1, else 0
            if pred.loc[index,column]==act_val:
                new_df.loc[index,column]=1
            else:
                new_df.loc[index,column]=0
    #In a rolling window of 8 quarters, determine the best forecaster of the last 8 quarters            
    i=0
    predictions={}
    for index,row in new_df.iterrows():
        #First value is after 8th quarter
        if i>=8:
            #sum all correct prediction over last 8 quarters
            best_list=[]
            best_values_series=new_df.iloc[(i-8):i].sum().sort_values(ascending=False)
            #list to check wheter there are further forecasters with the same amount of correct predictions
            for f in best_values_series.index:
                if best_values_series[f]==best_values_series[0]:
                    best_list.append(f)
            #collect the predictions of all forecasters with most amount of correct predictions        
            prediction_list=[]
            for forecaster in best_list:
                prediction_list.append(individual_forecasts.loc[index,forecaster])
            #take mean of all predictions
            predictions[index]=np.nanmean(prediction_list)
        i+=1
    #create DataFrame with best predictions
    best_predictions=pd.DataFrame(predictions,index=["Individual"]).T
    return(best_predictions)


In [9]:
best_predictions_TMTUI=qn_predictions(TMTUI,"2Y",TUY,1)
best_predictions_SMTUI=qn_predictions(SMTUI,"2Y",TUY,2)
best_predictions_TWTUI=qn_predictions(TWTUI,"2Y",TUY,4)
best_predictions_TMTYI=qn_predictions(TMTYI,"10Y",TYY,1)
best_predictions_SMTYI=qn_predictions(SMTYI,"10Y",TYY,2)
best_predictions_TWTYI=qn_predictions(TWTYI,"10Y",TYY,4)

In [10]:
def filesaver(df,docname):
    #Save file using relative path
    relative_path = os.path.join('..', '..',"Data", 'Consolidated',"Forecasts","IndividualForecasts","FinalPrediction",docname)
    df.to_csv(relative_path)

In [11]:
filesaver(best_predictions_TMTUI,"TMTUI.csv")
filesaver(best_predictions_SMTUI,"SMTUI.csv")
filesaver(best_predictions_TWTUI,"TWTUI.csv")
filesaver(best_predictions_TMTYI,"TMTYI.csv")
filesaver(best_predictions_SMTYI,"SMTYI.csv")
filesaver(best_predictions_TWTYI,"TWTYI.csv")